In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Layer, Dense, Activation

import import_ipynb
from my_layers import Attention, Average, WeightedSum, WeightedAspectEmb, MaxMargin

In [ ]:
def create_model(args, maxlen, vocab):
    
    @tf.keras.utils.register_keras_serializable(package='Custom', name='l1')
    def ortho_reg(weight_matrix):
        ### orthogonal regularization for aspect embedding matrix ###
        w_n = weight_matrix / K.cast(K.epsilon() + K.sqrt(K.sum(K.square(weight_matrix), axis=-1, keepdims=True)), K.floatx())
#         reg = K.sum(K.square(K.dot(w_n, K.transpose(w_n)) - K.eval(K.eye(w_n.shape[0]))))
#         reg = K.sum(K.square(K.dot(w_n, K.transpose(w_n)) - K.eye(w_n.shape[0])))
#         print(reg)
#         return args['ortho_reg']*reg
#         print(weight_matrix)
        return (0.01 * tf.math.reduce_sum(tf.math.abs(weight_matrix)))

    vocab_size = len(vocab)
    ##### Inputs #####

    sentence_input = Input(shape=(maxlen,), dtype='int32', name='sentence_input')
    neg_input = Input(shape=(args['neg_size'], maxlen), dtype='int32', name='neg_input')

    ##### Construct word embedding layer #####
    word_emb = Embedding(vocab_size, args['emb_dim'], mask_zero=True, name='word_emb')

    ##### Compute sentence representation #####
    e_w = word_emb(sentence_input)
    y_s = Average()(e_w)
    att_weights = Attention(name='att_weights')([e_w, y_s])
    z_s = WeightedSum()([e_w, att_weights])

    ##### Compute representations of negative instances #####
    e_neg = word_emb(neg_input)
    z_n = Average()(e_neg)

    ##### Reconstruction #####
    p_t = Dense(args['aspect_size'])(z_s)
    p_t = Activation('softmax', name='p_t')(p_t)
#     r_s = WeightedAspectEmb(args['aspect_size'], args['emb_dim'], name='aspect_emb',
#             kernel_regularizer=tf.keras.regularizers.L1(0.1))(p_t)
    
    r_s = WeightedAspectEmb(args['aspect_size'], args['emb_dim'], name='aspect_emb',
            kernel_regularizer=ortho_reg)(p_t)


    ##### Loss #####
    loss = MaxMargin(name='max_margin')([z_s, z_n, r_s])
    model = Model(inputs=[sentence_input, neg_input], outputs=loss)

    ### Word embedding and aspect embedding initialization ######
    if args['emb_path']:
        from w2vEmbReader import W2VEmbReader as EmbReader
        emb_reader = EmbReader(args['emb_path'], emb_dim=args['emb_dim'])
        model.get_layer('word_emb').embeddings_initializer = [emb_reader.get_emb_matrix_given_vocab(
                            vocab, model.get_layer('word_emb').get_weights()[0])]

        model.get_layer('aspect_emb').set_weights([emb_reader.get_aspect_matrix(args['aspect_size'])])

    return model